# Trove newspapers: data dashboard

Trove's collection of digitised newspapers is always changing – new articles and newspapers are being added by the NLA, while Trove users are busy correcting OCRd text and adding tags and comments. The search you run today might produce different results than the same query did a month ago, a year ago, or ten years ago. Researchers need to understand how these changes affect the queries they make, the results they find, and the arguments they construct. This dashboard helps researchers understand the context of their queries by presenting a snapshot of Trove's digitised newspapers, based on [weekly data harvests](https://github.com/wragge/trove-newspaper-totals). It shows Trove's current make up from a number of angles, as well as highlighting recent changes. It is updated every Sunday.

<ul class="browser-default">
    <li><a href="#Total-articles-and-user-activity">Total articles and user activity</a></li>
    <li><a href="#Total-articles-by-publication-year">Total articles by publication year</a></li>
    <li><a href="#Total-articles-by-publication-year-and-state">Total articles by publication year and state</a></li>
    <li><a href="#Article-categories">Article categories</a></li>
    <li><a href="#Newspaper-titles">Newspaper titles</a></li>
    <li><a href="#Total-articles-by-publication-state">Total articles by publication state</a></li>
    <li><a href="#Significant-events">Significant events</a></li>
</ul>

In [ ]:
import git
import time
import pandas as pd
from datetime import datetime
from io import BytesIO
import altair as alt
from IPython.display import display, HTML

In [ ]:
repo = git.Repo()

def get_versions(path, dates=[]):
    revlist = (
        (commit, (commit.tree / path).data_stream.read())
        for commit in repo.iter_commits(paths=path)
    )
    totals = []
    for commit, file in revlist:
        commit_date = datetime.fromtimestamp(commit.committed_date)
        df = pd.read_csv(BytesIO(file), parse_dates=dates)
        #df['date'] = commit_date
        #df = df.set_index('harvest_type').T
        #df = df.rename_axis(None, axis=1)
        # df.replace({'all': 'All articles', 'has:corrections': 'Articles with corrections', 'has:tags': 'Articles with tags', 'has:comments': 'Articles with comments'}, inplace=True)
        df['date'] = commit_date
        totals.append(df)
    return pd.concat(totals, ignore_index=True).sort_values('date')

def dhtml(string):
    display(HTML(string))
    
def make_clickable(val):
    return f'<a target="_blank" href="{val}">{val}</a>'

def left_align_columns(df):
    df = df.set_table_styles([{'selector': 'th', 'props': 'text-align:left'}, {'selector': '', 'props': 'table-layout:auto !important;'} ])
    return df.set_properties(**{'text-align': 'left'})

def left_align_headers(df):
    return df.set_table_styles([{'selector': '.row_heading', 'props': 'text-align:left'}, {'selector': 'table', 'props': 'table-layout:auto !important;'} ])

def current_total(df, col='harvest_type', val='All articles'):
    return df.loc[df[col] == val].iloc[-1]['total']

# Change in the last week
def change_in_total(df, col='harvest_type', val='All articles', period=1):
    return df.loc[df[col] == val]['total'].diff(periods=period).iloc[-1]

def harvest_date(timestamp):
    return pd.Timestamp(timestamp).strftime("%d %b %Y")

In [ ]:
dhtml(f'Last updated: {datetime.now().strftime("%A, %-d %B %Y")}')

In [ ]:
all_path = "data/total_articles_by_activity.csv"
df_all = get_versions(all_path)
df_all.replace({'all': 'All articles', 'has:corrections': 'Articles with corrections', 'has:tags': 'Articles with tags', 'has:comments': 'Articles with comments'}, inplace=True)
harvests_all = list(df_all['date'].unique())

----

## Total articles and user activity

In [ ]:
all_url = f'https://github.com/wragge/trove-newspaper-totals/blob/master/{all_path}'
dates = [{
    'Latest data harvest': f'{df_all["date"].max():%d %B %Y}',
    'First data harvest': f'{df_all["date"].min():%d %B %Y}',
    'Total harvests': len(df_all['date'].unique()),
    'Data': f'<a href={all_url} target="_blank">{all_url}</a>'
}]

left_align_columns(pd.DataFrame(dates).T.style.hide(axis=1))

In [ ]:
dhtml(f'<div class="alert alert-block alert-info"><h3>As of <b>{df_all["date"].max():%d %B %Y}</b> there are <b>{current_total(df_all, val="All articles"):,}</b> digitised newspaper articles in Trove.</h3></div>')

In [ ]:
harvest_types = ['All articles', 'Articles with corrections', 'Articles with tags', 'Articles with comments']

changes = []
for htype in harvest_types:
    changes.append({
        'Current total': f'{current_total(df_all, val=htype):,}',
        f'Change since {harvest_date(harvests_all[-2])}': f'{int(change_in_total(df_all, val=htype, period=1)):,}',
        f'Change since {harvest_date(harvests_all[-5])}': f'{int(change_in_total(df_all, val=htype, period=4)):,}',
        f'Change since {harvest_date(harvests_all[0])}': f'{int(change_in_total(df_all, val=htype, period=len(df_all["date"].unique())-1)):,}',
        'Percentage of total': f'{current_total(df_all, val=htype)/current_total(df_all):.2%}'
    })
    
left_align_headers(pd.DataFrame(changes, index=harvest_types).style)

In [ ]:
alt.Chart(df_all).mark_line(point=True).encode(
    x=alt.X('date:T', title=None, axis=alt.Axis(format='%e %b')),
    y=alt.Y('total:Q', title=None, scale=alt.Scale(zero=False)),
    tooltip=[alt.Tooltip('date:T', title='harvest date', format='%e %b %Y'), alt.Tooltip('total:Q', title='number of articles', format=','),],
    facet=alt.Facet('harvest_type:N', columns=2, title=None),
    color='harvest_type:N'
).resolve_scale(
    x='independent', 
    y='independent'
).properties(width=300, height=200)

----

## Total articles by publication year

In [ ]:
years_path = "data/total_articles_by_year.csv"
df_years = get_versions(years_path)

In [ ]:
years_url = f'https://github.com/wragge/trove-newspaper-totals/blob/master/{years_path}'
dates = [{
    'Latest data harvest': f'{df_years["date"].max():%d %B %Y}',
    'First data harvest': f'{df_years["date"].min():%d %B %Y}',
    'Total harvests': len(df_years['date'].unique()),
    'Data': f'<a href={years_url} target="_blank">{years_url}</a>'
}]

left_align_columns(pd.DataFrame(dates).T.style.hide(axis=1))

This chart shows the total number of digitised newspaper articles by year of publication. The distribution will change over time as digitisation priorities shift and more newspapers are added. This chart is useful in understanding how digitisation priorities and copyright restraints have shaped the total newspaper corpus. 

In [ ]:
alt.Chart(df_years.loc[df_years['date'] == df_years['date'].max()]).mark_bar(opacity=1).encode(
    x=alt.X('year:O', axis=alt.Axis(labelOverlap=True), title='year of publication'),
    y=alt.Y('total:Q', title='number of articles'),
    tooltip=[alt.Tooltip('year:O', title='year of publication'), alt.Tooltip('total:Q', format=",", title='number of articles')],
).properties(width=800)

----

## Total articles by publication year and state

In [ ]:
states_years_path = "data/total_articles_by_year_and_state.csv"
df_states_years = get_versions(states_years_path)
harvests_states_years = list(df_states_years['date'].unique())
df_states = df_states_years.loc[df_states_years['date'] == df_states_years['date'].max()].groupby('state')['total'].sum().to_frame().reset_index()
df_states_all = df_states_years.groupby(['state', 'date'])['total'].sum().to_frame().reset_index()

In [ ]:
states_years_url = f'https://github.com/wragge/trove-newspaper-totals/blob/master/{states_years_path}'
dates = [{
    'Latest data harvest': f'{df_states_years["date"].max():%d %B %Y}',
    'First data harvest': f'{df_states_years["date"].min():%d %B %Y}',
    'Total harvests': len(df_states_years['date'].unique()),
    'Data': f'<a href={states_years_url} target="_blank">{states_years_url}</a>'
}]

left_align_columns(pd.DataFrame(dates).T.style.hide(axis=1))

This interactive chart helps you explore the distribution of articles by both publication year and state. Click on a state in the bar chart or legend to filter the results by year. Click anywhere in the background of the chart to reset. This enables you to compare digitisation patterns in different states and see how they contribute to the whole corpus.

In [ ]:
# Use color as the selector for filtering the chart
selection = alt.selection_multi(encodings=["color"])

# Color is based on the state, or gray if another state is selected
color = alt.condition(
    selection, alt.Color("state:N", legend=None), alt.value("lightgray")
)

# A basic area chart, starts stacked, but when filtered shows only the active state
area = (
    alt.Chart(df_states_years.loc[df_states_years['date'] == df_states_years['date'].max()])
    .mark_bar()
    .encode(
        # Years on the X axis
        x=alt.X("year:O", axis=alt.Axis(labelOverlap=True, format="c", title="year of publication")),
        # Number of articles on the Y axis
        y=alt.Y(
            "total:Q",
            axis=alt.Axis(format=",d", title="number of articles"),
            stack=True,
        ),
        # Color uses the settings defined above
        color=color,
        # Details on hover
        tooltip=[
            "state",
            alt.Tooltip("year:Q", title="year of publication"),
            alt.Tooltip("total:Q", title="number of articles", format=","),
        ],
    )
    .properties(width=800, height=400)
    .transform_filter(
        # Filter data by state when a state is selected
        selection
    )
    .add_selection(selection)
)

# Add a bar chart showing the number of articles per state
bar = (
    alt.Chart(df_states)
    .mark_bar()
    .encode(
        # State on the X axis
        x=alt.X("state:N", title="state"),
        # Number of articles on the Y axis
        y=alt.Y(
            "total:Q", axis=alt.Axis(format=",d", title="number of articles")
        ),
        # Details on hover
        tooltip=[
            alt.Tooltip("state", title="state"),
            alt.Tooltip("total:Q", title="number of articles", format=","),
        ],
        # Color based on state as defined above
        color=color,
    )
    .properties(width=800, height=150)
    .add_selection(
        # Highlight state when selected
        selection
    )
)

# For good measure we'll add an interactive legend (which is really just a mini chart)
# This makes it easier to select states that don't have many articles
legend = (
    alt.Chart(df_states)
    .mark_rect()
    .encode(
        # Show the states
        y=alt.Y("state:N", axis=alt.Axis(orient="right", title=None)),
        # Color as above
        color=color,
    )
    .add_selection(
        # Highlight on selection
        selection
    )
)

# Concatenate the charts -- area & legend hotizontal, then bar added vertically
(area | legend) & bar

In [ ]:
dhtml(f'<h3>Total articles added since {harvest_date(harvests_states_years[-2])} by publication year and state</h3>')

This chart shows the total number of digitised newspaper articles added since the previous harvest by year and place of publication. It's useful in seeing how search results from particular time periods and locations might be affected by current digitisation priorities.

In [ ]:
year_changes = []
for g, d in df_states_years.groupby(['year', 'state']):
    change = d.sort_values('date')['total'].diff(periods=1).iloc[-1]
    year_changes.append({'year': g[0], 'state': g[1], 'change': change})
df_year_changes1 = pd.DataFrame(year_changes)

alt.Chart(df_year_changes1.loc[df_year_changes1['change'] >= 0]).mark_bar(opacity=1).encode(
    x=alt.X('year:O', axis=alt.Axis(labelOverlap=True), title='year of publication'),
    y=alt.Y('change:Q', stack=True, title='increase in articles'),
    color='state:N',
    tooltip=['state:N', alt.Tooltip('year:O', title='year of publication'), alt.Tooltip('change:Q', title='increase in articles', format=',')]
).properties(width=800)

In [ ]:
dhtml(f'<h3>Total articles added since {harvest_date(harvests_states_years[0])} by publication year and state</h3>')

This chart shows the total number of digitised newspaper articles added since the first harvest by year and place of publication. It's useful in seeing how search results from particular time periods or locations might be affected by recent digitisation priorities.

In [ ]:
df_years_start_end = df_states_years.loc[(df_states_years['date'] == df_states_years['date'].max()) | (df_states_years['date'] == df_states_years['date'].min())]

year_changes = []
for g, d in df_years_start_end.groupby(['year', 'state']):
    change = d.sort_values('date')['total'].diff().iloc[-1]
    year_changes.append({'year': g[0], 'state': g[1], 'change': change})
df_year_changes_all = pd.DataFrame(year_changes)

alt.Chart(df_year_changes_all.loc[df_year_changes_all['change'] >= 0]).mark_bar(opacity=1).encode(
    x=alt.X('year:O', axis=alt.Axis(labelOverlap=True), title='year of publication'),
    y=alt.Y('change:Q', stack=True, title='increase in articles'),
    color='state:N',
    tooltip=['state:N', alt.Tooltip('year:O', title='year of publication'), alt.Tooltip('change:Q', title='increase in articles', format=',')]
).properties(width=800)

----

## Article categories

In [ ]:
cat_path = "data/total_articles_by_category.csv"
df_cat = get_versions(cat_path)
harvests_cat = list(df_cat['date'].unique())

In [ ]:
cat_url = f'https://github.com/wragge/trove-newspaper-totals/blob/master/{cat_path}'
dates = [{
    'Latest data harvest': f'{df_cat["date"].max():%d %B %Y}',
    'First data harvest': f'{df_cat["date"].min():%d %B %Y}',
    'Total harvests': len(df_cat['date'].unique()),
    'Data': f'<a href={cat_url} target="_blank">{cat_url}</a>'
}]
left_align_columns(pd.DataFrame(dates).T.style.hide(axis=1))

In [ ]:
df_cat_current = df_cat.loc[df_cat['date'] == df_cat['date'].max()]

alt.Chart(df_cat_current).mark_bar().encode(
    x=alt.X('total:Q', scale=alt.Scale(type="log"), title='number of articles (log scale)'),
    y='category:N',
    tooltip=['category', alt.Tooltip('total:Q', format=',', title='number of articles')],
    color=alt.Color('category:N', scale=alt.Scale(scheme='category20'))
).properties(width=800)

In [ ]:
categories = sorted(list(df_cat['category'].unique()))

changes = []
for cat in categories:
    changes.append({
        'Current total': f'{current_total(df_cat, col="category", val=cat):,}',
        f'Change since {harvest_date(harvests_cat[-2])}': f'{int(change_in_total(df_cat, col="category", val=cat, period=1)):,}',
        f'Change since {harvest_date(harvests_cat[-5])}': f'{int(change_in_total(df_cat, col="category", val=cat, period=4)):,}',
        f'Change since {harvest_date(harvests_cat[0])}': f'{int(change_in_total(df_cat, col="category", val=cat, period=len(df_cat["date"].unique())-1)):,}',
        'Percentage of total': f'{current_total(df_cat, col="category", val=cat)/current_total(df_all):.3%}'
    })
    
left_align_headers(pd.DataFrame(changes, index=categories).style)

----

## Newspaper titles

In [ ]:
news_path = "data/total_articles_by_newspaper.csv"
df_news = get_versions("data/total_articles_by_newspaper.csv", dates=['start_date', 'end_date'])
harvests_news = list(df_news['date'].unique())

In [ ]:
harvests = pd.DataFrame(df_news['date'].unique(), columns=['date'])

In [ ]:
news_url = f'https://github.com/wragge/trove-newspaper-totals/blob/master/{news_path}'
dates = [{
    'Latest data harvest': f'{harvests["date"].max():%d %B %Y}',
    'First data harvest': f'{harvests["date"].min():%d %B %Y}',
    'Total harvests': harvests.shape[0],
    'Data': f'<a href={news_url} target="_blank">{news_url}</a>'
}]
left_align_columns(pd.DataFrame(dates).T.style.hide(axis=1))

In [ ]:
dhtml(f'<div class="alert alert-block alert-info"><h3>As of <b>{df_news["date"].max():%d %B %Y}</b> there are <b>{len(df_news["title_id"].unique()):,}</b> different newspapers in Trove.</h3></div>')

In [ ]:
news_summary = [{
    'Total newspapers': f'{len(df_news["title_id"].unique()):,}',
    'Earliest publication date': f'{df_news["start_date"].min():%d %B %Y}',
    'Latest publication date': f'{df_news["end_date"].max():%d %B %Y}'
}]
left_align_columns(pd.DataFrame(news_summary).T.style.hide(axis=1))

### Newspapers by state of publication

This chart shows the place of publication of newspaper titles in Trove.

In [ ]:
alt.Chart(df_news.loc[df_news['date'] == df_news['date'].max()]).mark_bar().encode(
    y='state:N',
    x=alt.X('count():Q', title='number of newspapers'),
    tooltip=['state:N', alt.Tooltip('count():Q', title='number of newspapers')],
    color=alt.Color('state:N')
)

### Changes in the number of newspapers by state of publication

In [ ]:
df_news_states = df_news.groupby(['state', 'date']).count().reset_index()
df_news_states = df_news_states[['state', 'date', 'total']]

In [ ]:
states = sorted(list(df_news['state'].dropna().unique()))

changes = []
for state in states:
    changes.append({
        'Current total': f'{current_total(df_news_states, col="state", val=state):,}',
        f'Change since {harvest_date(harvests_news[-2])}': f'{int(change_in_total(df_news_states, col="state", val=state, period=1)):,}',
        f'Change since {harvest_date(harvests_news[-5])}': f'{int(change_in_total(df_news_states, col="state", val=state, period=4)):,}',
        f'Change since {harvest_date(harvests_news[0])}': f'{int(change_in_total(df_news_states, col="state", val=state, period=len(df_news_states["date"].unique())-1)):,}',
        'Percentage of total': f'{current_total(df_news_states, col="state", val=state)/len(df_news["title"].unique()):.2%}'
    })
    
left_align_headers(pd.DataFrame(changes, index=states).style)

In [ ]:
df_news_latest = df_news.loc[df_news['date'] == harvests.iloc[-1]['date']][['title_id', 'title', 'state', 'total']]
df_news_minus1 = df_news.loc[df_news['date'] == harvests.iloc[-2]['date']][['title_id', 'title', 'state']]
df_news_minus4 = df_news.loc[df_news['date'] == harvests.iloc[-4]['date']][['title_id', 'title', 'state']]
df_news_all = df_news.loc[df_news['date'] == harvests.iloc[0]['date']][['title_id', 'title', 'state']]

In [ ]:
dhtml(f'<h3>Newspapers added since {harvest_date(harvests_news[-2])}</h3>')

In [ ]:
# Find titles added since last harvest
news_changes_1 = pd.concat([df_news_latest,df_news_minus1]).drop_duplicates(subset=['title_id'], keep=False).dropna()
if news_changes_1.empty:
    dhtml('<ul class="browser-default"><li>No newspapers added in this period.</li></ul>')
else:
    news_changes_1['link'] = news_changes_1['title_id'].apply(lambda x: f'https://nla.gov.au/nla.news-title{x}')
    display(left_align_columns(news_changes_1.style.format({'link': make_clickable}).hide()))

In [ ]:
dhtml(f'<h3>Newspapers added since {harvest_date(harvests_news[-5])}</h3>')

In [ ]:
news_changes_4 = pd.concat([df_news_latest,df_news_minus4]).drop_duplicates(subset=['title_id'], keep=False).dropna()
if news_changes_4.empty:
    dhtml('<ul class="browser-default"><li>No newspapers added in this period.</li></ul>')
else:
    news_changes_4['link'] = news_changes_4['title_id'].apply(lambda x: f'https://nla.gov.au/nla.news-title{x}')
    display(left_align_columns(news_changes_4.style.format({'link': make_clickable}).hide()))

In [ ]:
def newspaper_total_changes(df, period=1):
    news_changes = []
    for g, n in df.groupby('title_id'):
        change = n['total'].diff(periods=period).iloc[-1]
        if change > 0:
            news_changes.append({
                'title_id': g,
                'title': n.iloc[-1]['title'],
                'change in articles': int(change),
                'link': f'https://nla.gov.au/nla.news-title{g}'
            })
    return pd.DataFrame(news_changes)

In [ ]:
dhtml(f'<h3>Changes in the number of articles from newspapers since {harvest_date(harvests_news[-2])}</h3>')

In [ ]:
news_articles_1 = newspaper_total_changes(df_news, 1)
if news_articles_1.empty:
    dhtml('<ul class="browser-default"><li>No changes in this period.</li></ul>')
else:
    news_articles_1['link'] = news_articles_1['title_id'].apply(lambda x: f'https://nla.gov.au/nla.news-title{x}')
    display(left_align_columns(news_articles_1.style.format({'link': make_clickable, 'change in articles': '{:,}'}).hide()))

In [ ]:
dhtml(f'<h3>Changes in the number of articles from newspapers since {harvest_date(harvests_news[-5])}</h3>')

In [ ]:
news_articles_4 = newspaper_total_changes(df_news, 4)
if news_articles_4.empty:
    dhtml('<ul class="browser-default"><li>No changes in this period.</li></ul>')
else:
    news_articles_4['link'] = news_articles_4['title_id'].apply(lambda x: f'https://nla.gov.au/nla.news-title{x}')
    display(left_align_columns(news_articles_4.style.format({'link': make_clickable, 'change in articles': '{:,}'}).hide()))

----

## Total articles by publication state

Data about the number of articles by publication place (state) is available both from the main `result` API using the `state` facet, and from the `newspaper` API. As noted under [Significant events](#Significant-events), these totals don't always agree. I've included both below, so you can compare.

### Number of articles by state of publication (via `result` API)

In [ ]:
alt.Chart(df_states).mark_bar().encode(
        # State on the X axis
        y=alt.Y("state:N", title="state"),
        # Number of articles on the Y axis
        x=alt.X(
            "total:Q", axis=alt.Axis(format=",d", title="number of articles")
        ),
        # Details on hover
        tooltip=[
            alt.Tooltip("state", title="state"),
            alt.Tooltip("total:Q", title="number of articles", format=","),
        ],
        # Color based on state as defined above
        color='state:N',
    ).properties()


### Number of articles by state of publication (via `newspaper` API)

In [ ]:
df_news_states_totals = df_news_latest.groupby(['state'])['total'].sum().reset_index()
alt.Chart(df_news_states_totals).mark_bar().encode(
        # State on the X axis
        y=alt.Y("state:N", title="state"),
        # Number of articles on the Y axis
        x=alt.X(
            "total:Q", axis=alt.Axis(format=",d", title="number of articles")
        ),
        # Details on hover
        tooltip=[
            alt.Tooltip("state", title="state"),
            alt.Tooltip("total:Q", title="number of articles", format=","),
        ],
        # Color based on state as defined above
        color='state:N',
    ).properties()


### Changes in the number of articles by state of publication (via `result` API)

In [ ]:
states = sorted(list(df_states_all['state'].unique()))

changes = []
for state in states:
    changes.append({
        'Current total': f'{current_total(df_states_all, col="state", val=state):,}',
        f'Change since {harvest_date(harvests_states_years[-2])}': f'{int(change_in_total(df_states_all, col="state", val=state, period=1)):,}',
        f'Change since {harvest_date(harvests_states_years[-5])}': f'{int(change_in_total(df_states_all, col="state", val=state, period=4)):,}',
        f'Change since {harvest_date(harvests_states_years[0])}': f'{int(change_in_total(df_states_all, col="state", val=state, period=len(df_states_all["date"].unique())-1)):,}',
        'Percentage of total': f'{current_total(df_states_all, col="state", val=state)/current_total(df_all):.3%}'
    })
    
left_align_headers(pd.DataFrame(changes, index=states).style)

### Changes in the number of articles by state of publication (via `newspaper` API)

In [ ]:
df_news_states_all = df_news.groupby(['state', 'date'])['total'].sum().reset_index()
states = sorted(list(df_news_states['state'].unique()))

changes = []
for state in states:
    changes.append({
        'Current total': f'{current_total(df_news_states_all, col="state", val=state):,}',
        f'Change since {harvest_date(harvests_states_years[-2])}': f'{int(change_in_total(df_news_states_all, col="state", val=state, period=1)):,}',
        f'Change since {harvest_date(harvests_states_years[-5])}': f'{int(change_in_total(df_news_states_all, col="state", val=state, period=4)):,}',
        f'Change since {harvest_date(harvests_states_years[0])}': f'{int(change_in_total(df_news_states_all, col="state", val=state, period=len(df_states_all["date"].unique())-1)):,}',
        'Percentage of total': f'{current_total(df_news_states_all, col="state", val=state)/current_total(df_all):.3%}'
    })
    
left_align_headers(pd.DataFrame(changes, index=states).style)

----

## Significant events

While the changes documented here generally reflect the addition of new articles and newspapers, the total numbers can also be affected by system configuration and optimisation events, such as rebuilding the index. It's difficult to highlight these sorts of anomalies in the charts above, but where I notice them I'll add a note here.

### 31 May 2022 – reindex?

Between the harvests on 29 May and 5 June 2022 there was a dramatic change in the distribution of articles by state (using the state facet). The number of articles from NSW decreased by 939,065, and the number of articles from Victoria increased by 961,387. There were also some smaller changes affecting other states. There was a scheduled maintenance window on 31 May, which makes me think that these reflect changes in the underlying search index, rather than the actual number of articles. Further evidence for this is the fact that the number of articles per state calculated by using the `newspaper` API endpoint were out-of-sync with the article totals until the end of May. Since then, both sets of article totals have remained in sync. So, in short, the number of articles returned by the state facet between 19 April 2022 (when my harvests started) and 31 May 2022 can't be trusted.

----

Created by [Tim Sherratt](https://timsherratt.org/) for the [GLAM Workbench](https://glam-workbench.net/). Support this project by becoming a [GitHub sponsor](https://github.com/sponsors/wragge?o=esb).